# Extracting Features with ViT
- https://huggingface.co/docs/transformers/model_doc/vit#vision-transformer-vit
- https://arxiv.org/abs/2010.11929

In [3]:
# import os
import torch
# import csv

import pandas as pd
import numpy as np
# import torchvision.transforms as transforms

from PIL import Image
from transformers import ViTImageProcessor, ViTFeatureExtractor, ViTModel
# from sklearn.model_selection import train_test_split, GridSearchCV

2023-04-25 08:54:56.411983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
BASE = '/Users/brinkley97/Documents/development/'
CLASS_PATH = 'classes/csci_535_multimodal_probabilistic_learning/'
DATASET_PATH = 'datasets/'

In [5]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

In [6]:
def load_data(file):
    original_data = pd.read_csv(file)
    # original_data = pd.DataFrame(file)
    copy_of_data = original_data.copy()
    return copy_of_data

In [7]:
file_paths = BASE + CLASS_PATH + DATASET_PATH + 'cremaD_mspR_mspT.csv'

In [8]:
dataset_paths_copy = load_data(file_paths)
dataset_paths_copy

,Unnamed: 0,CREMA-D Paths,MSP R Paths,MSP T Paths
0,0,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...
1,1,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...
2,2,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...
3,3,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...
4,4,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...,/Users/brinkley97/Documents/development/classe...
...,...,...,...,...
114818,114818,/Users/brinkley97/Documents/development/classe...,NaN,NaN
114819,114819,/Users/brinkley97/Documents/development/classe...,NaN,NaN
114820,114820,/Users/brinkley97/Documents/development/classe...,NaN,NaN
114821,114821,/Users/brinkley97/Documents/development/classe...,NaN,NaN


# MSP

In [9]:
MSP_DATASET_PATH = 'msp/videos/r_and_t_frames/'

## R

In [10]:
msp_r_paths = dataset_paths_copy['MSP R Paths']
# type(msp_r_paths), msp_r_paths

In [11]:
sub_set_msp_r_paths = msp_r_paths.loc[600:900]
sub_set_msp_r_paths

600    /Users/brinkley97/Documents/development/classe...
601    /Users/brinkley97/Documents/development/classe...
602    /Users/brinkley97/Documents/development/classe...
603    /Users/brinkley97/Documents/development/classe...
604    /Users/brinkley97/Documents/development/classe...
                             ...                        
896    /Users/brinkley97/Documents/development/classe...
897    /Users/brinkley97/Documents/development/classe...
898    /Users/brinkley97/Documents/development/classe...
899    /Users/brinkley97/Documents/development/classe...
900    /Users/brinkley97/Documents/development/classe...
Name: MSP R Paths, Length: 301, dtype: object

In [12]:
# msp_r_df = pd.DataFrame(sub_set_msp_r_paths, columns=['MSP R Paths'])
# msp_r_df

In [13]:
x = []
    
for msp_r_path_idx in range(len(sub_set_msp_r_paths)):
    # print(msp_r_path_idx)
    msp_r_path = msp_r_paths[msp_r_path_idx]
    
    specific_frame = Image.open(msp_r_path)
    # print(np.shape(specific_frame))

    inputs = processor(images=specific_frame, return_tensors="pt")
    x.append(inputs)

In [ ]:
# x

In [14]:
import pickle

# Input

In [15]:
sub_set_msp_r = BASE + CLASS_PATH + DATASET_PATH + 'subsets_vit/' + 'inputs-mspR_subset3.txt'
sub_set_msp_r

'/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/subsets_vit/inputs-mspR_subset3.txt'

In [16]:
with open(sub_set_msp_r, 'wb') as f:
    pickle.dump(x, f)  

In [17]:
reading_sub_set_msp_r = pd.read_pickle(sub_set_msp_r)

In [18]:
# reading_sub_set_msp_r

# Hidden

In [19]:
all_last_hidden_states = []

for specific_frame in range(len(reading_sub_set_msp_r)):
    inputs = reading_sub_set_msp_r[specific_frame]
    outputs = model(**inputs)
    
    # last_hidden_states == representation (1 and 2 with GradCam)
    last_hidden_states = outputs.last_hidden_state
    all_last_hidden_states.append(last_hidden_states)

In [24]:
# 'subsets_vit/'
msp_r_hidden_states = BASE + CLASS_PATH + DATASET_PATH + 'subsets_vit/' + 'hiddenStates-mspR_subset3.txt'

In [25]:
with open(msp_r_hidden_states, 'wb') as f:
    pickle.dump(all_last_hidden_states, f)  

In [26]:
reading_msp_r_hidden_states = pd.read_pickle(msp_r_hidden_states)

In [28]:
# reading_msp_r_hidden_states

In [29]:
extracted_features_per_video = torch.empty((0, 197, 768))
save_efs = []
for specific_frame_hs in range(len(reading_msp_r_hidden_states)):
    
    hs = reading_msp_r_hidden_states[specific_frame_hs]
    extracted_features_per_video = torch.vstack((extracted_features_per_video, hs))
    
    save_efs.append(extracted_features_per_video)

In [ ]:
# save_efs

In [30]:
sample_msp_r_extracted_features = BASE + CLASS_PATH + DATASET_PATH + 'subsets_vit/' + 'extractedFeatures-mspR_subset3.txt'

In [31]:
torch.save(save_efs, sample_msp_r_extracted_features)

RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 10334768960 vs 10334768856

In [20]:
fe_crema_d = torch.load(sample_msp_r_extracted_features)

In [21]:
# fe_crema_d

[tensor([[[-1.6990e-02,  1.5769e-02, -1.6725e-01,  ..., -4.3739e-01,
            3.8835e-02,  3.5149e-01],
          [-3.0596e-01,  3.4649e-04, -4.6682e-02,  ..., -2.2991e-01,
           -2.2290e-01,  4.4321e-01],
          [-3.2646e-01, -1.4196e-01, -6.6563e-02,  ..., -2.6051e-01,
           -1.1213e-01,  3.1158e-02],
          ...,
          [-3.1556e-02, -1.3323e-01, -1.8170e-01,  ..., -4.2107e-01,
            2.4766e-01,  2.6476e-01],
          [-2.5025e-01, -7.6298e-02, -9.1349e-02,  ..., -2.9831e-01,
           -1.3129e-01,  2.3660e-01],
          [-3.1241e-01, -1.4167e-01, -5.4306e-02,  ..., -2.4009e-01,
           -1.8457e-01,  2.9366e-01]]], requires_grad=True),
 tensor([[[-1.6990e-02,  1.5769e-02, -1.6725e-01,  ..., -4.3739e-01,
            3.8835e-02,  3.5149e-01],
          [-3.0596e-01,  3.4649e-04, -4.6682e-02,  ..., -2.2991e-01,
           -2.2290e-01,  4.4321e-01],
          [-3.2646e-01, -1.4196e-01, -6.6563e-02,  ..., -2.6051e-01,
           -1.1213e-01,  3.1158e-02],